In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import joblib # Added for saving scaler, though it was in cell 6 before

# --- Configuration ---
# Directory where your PROCESSED load and weather files are stored
processed_data_dir = r'D:\.Study\projects\EnergyForecasting\data\processed' # !!! UPDATE THIS PATH IF NEEDED !!!

# --- NEW FILENAMES (from your multi-year processing notebooks) ---
# Make sure these EXACTLY match the output filenames from your
# 02-load-data-processing and 03-weather-data-processing notebooks.
processed_load_filename_MULTIYEAR = 'pjm_hourly_load_multi_zone_cleaned_2022-2025May.csv'
processed_weather_filename_MULTIYEAR = 'pjm_hourly_weather_multi_zone_cleaned_2022-2025May.csv'

# Directory to save the final, model-ready datasets
model_ready_data_dir = r'D:\.Study\projects\EnergyForecasting\src\model_ready' # !!! UPDATE THIS PATH IF NEEDED !!!

# List of your PJM zones (ensure this is consistent with previous notebooks)
# Example: ['DOM', 'PN', 'PEPCO', 'AECO', 'PE']
PJM_ZONES_UPPER = ['DOM', 'PN', 'PEPCO', 'AECO', 'PE'] # !!! UPDATE TO MATCH YOUR ZONES !!!

# Define target columns (load for each zone)
target_column_template = '{ZONE}_Load'

print(f"Processed data directory: {processed_data_dir}")
print(f"Model-ready data directory: {model_ready_data_dir}")
print(f"Zones being processed: {PJM_ZONES_UPPER}")

Processed data directory: D:\.Study\projects\EnergyForecasting\data\processed
Model-ready data directory: D:\.Study\projects\EnergyForecasting\data\model_ready
Zones being processed: ['DOM', 'PN', 'PEPCO', 'AECO', 'PE']


In [2]:
# Construct full file paths using the new multi-year filenames
load_data_path = os.path.join(processed_data_dir, processed_load_filename_MULTIYEAR)
weather_data_path = os.path.join(processed_data_dir, processed_weather_filename_MULTIYEAR)

load_df = pd.DataFrame() # Initialize as empty
weather_df = pd.DataFrame() # Initialize as empty

# Load the datasets
try:
    load_df = pd.read_csv(load_data_path, index_col=0, parse_dates=True)
    print(f"Successfully loaded load data: {load_data_path}")
    print(f"  Load data shape: {load_df.shape}")
    # print(load_df.head(3)) # Keep for verification if needed
except FileNotFoundError:
    print(f"ERROR: Load data file not found at {load_data_path}")
    print("Please ensure 02-load-data-processing notebook has run successfully and saved the file with the correct name.")
except Exception as e:
    print(f"ERROR loading load data from {load_data_path}: {e}")


try:
    weather_df = pd.read_csv(weather_data_path, index_col=0, parse_dates=True)
    print(f"\nSuccessfully loaded weather data: {weather_data_path}")
    print(f"  Weather data shape: {weather_df.shape}")
    # print(weather_df.head(3)) # Keep for verification if needed
except FileNotFoundError:
    print(f"ERROR: Weather data file not found at {weather_data_path}")
    print("Please ensure 03-weather-data-processing notebook has run successfully and saved the file with the correct name.")
except Exception as e:
    print(f"ERROR loading weather data from {weather_data_path}: {e}")


# Ensure indices are timezone-aware (UTC) if they aren't already
# The previous processing steps should have handled this, but good to double-check.
if not load_df.empty:
    if load_df.index.tz is None:
        print("Localizing load_df index to UTC...")
        load_df.index = load_df.index.tz_localize('UTC', ambiguous='NaT', nonexistent='NaT')
    elif str(load_df.index.tz).upper() != 'UTC':
        print(f"Converting load_df index from {load_df.index.tz} to UTC...")
        load_df.index = load_df.index.tz_convert('UTC')
    else:
        print(f"Load_df index timezone is already UTC: {load_df.index.tz}")

if not weather_df.empty:
    if weather_df.index.tz is None:
        print("Localizing weather_df index to UTC...")
        weather_df.index = weather_df.index.tz_localize('UTC', ambiguous='NaT', nonexistent='NaT')
    elif str(weather_df.index.tz).upper() != 'UTC':
        print(f"Converting weather_df index from {weather_df.index.tz} to UTC...")
        weather_df.index = weather_df.index.tz_convert('UTC')
    else:
        print(f"Weather_df index timezone is already UTC: {weather_df.index.tz}")

Successfully loaded load data: D:\.Study\projects\EnergyForecasting\data\processed\pjm_hourly_load_multi_zone_cleaned_2022-2025May.csv
  Load data shape: (29879, 5)

Successfully loaded weather data: D:\.Study\projects\EnergyForecasting\data\processed\pjm_hourly_weather_multi_zone_cleaned_2022-2025May.csv
  Weather data shape: (29928, 55)
Localizing load_df index to UTC...
Weather_df index timezone is already UTC: UTC


In [3]:
combined_df = pd.DataFrame() # Initialize as empty

if not load_df.empty and not weather_df.empty:
    print("\n--- Aligning and Merging DataFrames ---")
    
    # Ensure both DataFrames have sorted indices before intersection for robustness
    load_df.sort_index(inplace=True)
    weather_df.sort_index(inplace=True)
    print("  Ensured load_df and weather_df indices are sorted.")

    # Find the intersection of the two DatetimeIndexes
    common_index = load_df.index.intersection(weather_df.index)
    print(f"  Number of common timestamps found: {len(common_index)}")
    
    if len(common_index) == 0:
        print("  ERROR: No common timestamps between load and weather data. Cannot merge.")
        print(f"    Load data index range: {load_df.index.min()} to {load_df.index.max()}")
        print(f"    Weather data index range: {weather_df.index.min()} to {weather_df.index.max()}")
    else:
        load_df_aligned = load_df.loc[common_index]
        weather_df_aligned = weather_df.loc[common_index]
        
        print(f"  Load data shape after alignment: {load_df_aligned.shape}")
        print(f"  Weather data shape after alignment: {weather_df_aligned.shape}")
        
        combined_df = pd.concat([load_df_aligned, weather_df_aligned], axis=1)
        
        print("\nCombined DataFrame (first 3 rows):")
        print(combined_df.head(3))
        print("\nInformation about Combined DataFrame:")
        combined_df.info()
        
        # Check for NaNs after merge (should ideally be 0 if common_index is used correctly)
        nans_after_merge = combined_df.isnull().sum().sum()
        if nans_after_merge > 0:
            print(f"\nTotal NaNs in combined_df after merge: {nans_after_merge}")
            print("  This might indicate issues if load/weather data had internal NaNs not filled, or column name clashes.")
            print("  Attempting ffill and bfill...")
            combined_df.ffill(inplace=True)
            combined_df.bfill(inplace=True)
            print(f"  Total NaNs after fill: {combined_df.isnull().sum().sum()}")
        else:
            print(f"\nNo NaNs in combined_df after merge.")
else:
    print("One or both DataFrames (load_df, weather_df) are empty. Cannot merge.")


--- Aligning and Merging DataFrames ---
  Ensured load_df and weather_df indices are sorted.
  Number of common timestamps found: 29879
  Load data shape after alignment: (29879, 5)
  Weather data shape after alignment: (29879, 55)

Combined DataFrame (first 3 rows):
                           DOM_Load   PN_Load  PEPCO_Load  AECO_Load  \
2022-01-01 05:00:00+00:00  9879.540  1517.266    2169.749    871.977   
2022-01-01 06:00:00+00:00  9549.307  1462.484    2065.314    834.028   
2022-01-01 07:00:00+00:00  9346.150  1434.579    2013.883    796.671   

                            PE_Load  DOM_Temp_C  DOM_DewPoint_C  \
2022-01-01 05:00:00+00:00  3490.865        -7.8           -14.4   
2022-01-01 06:00:00+00:00  3352.814        -6.0           -15.3   
2022-01-01 07:00:00+00:00  3224.528        -4.7           -15.1   

                           DOM_AppTemp_C  DOM_Precip_mm  DOM_WeatherCode  ...  \
2022-01-01 05:00:00+00:00          -12.4            0.0              3.0  ...   
2022-01-01 

In [4]:
if not combined_df.empty:
    print("\n--- Feature Engineering: Adding Time-Based Features ---")
    
    if not isinstance(combined_df.index, pd.DatetimeIndex):
        print("ERROR: combined_df.index is not a DatetimeIndex. Cannot create time features.")
    else:
        # Basic time features
        combined_df['hour_of_day'] = combined_df.index.hour
        combined_df['day_of_week'] = combined_df.index.dayofweek # Monday=0, Sunday=6
        combined_df['day_of_year'] = combined_df.index.dayofyear
        combined_df['month'] = combined_df.index.month
        combined_df['week_of_year'] = combined_df.index.isocalendar().week.astype(int)
        combined_df['year'] = combined_df.index.year
        print("Added basic time features (hour, day, month, year, etc.).")

        # --- ADDING LAGGED LOAD FEATURES ---
        print("\nAdding lagged load features...")
        lag_periods_hours = {
            'lag_24h': 24,        # Previous day, same hour
            'lag_48h': 48,        # Two days ago, same hour
            'lag_168h': 24 * 7    # Previous week, same hour
        }
        for zone in PJM_ZONES_UPPER:
            load_col_name = target_column_template.replace('{ZONE}', zone)
            if load_col_name in combined_df.columns:
                for lag_name, hours in lag_periods_hours.items():
                    combined_df[f'{load_col_name}_{lag_name}'] = combined_df[load_col_name].shift(hours)
                print(f"  Added lagged features for {load_col_name}")
            else:
                print(f"  WARNING: Load column {load_col_name} not found for adding lags.")
        
        # --- ADDING ROLLING MEAN WEATHER FEATURES (Example for Temperature) ---
        # This adds complexity but can be useful. You can expand to other weather vars.
        print("\nAdding rolling mean weather features (e.g., Temp_C)...")
        rolling_window_hours = 3 # 3-hour rolling average
        for zone in PJM_ZONES_UPPER:
            temp_col_name = f"{zone}_Temp_C" # Assuming this is your temp column name structure
            if temp_col_name in combined_df.columns:
                combined_df[f'{temp_col_name}_roll_mean_{rolling_window_hours}h'] = combined_df[temp_col_name].rolling(window=rolling_window_hours, min_periods=1).mean()
                # Shift by 1 because rolling mean includes current hour; we want mean of *past* hours as feature
                combined_df[f'{temp_col_name}_roll_mean_{rolling_window_hours}h'] = combined_df[f'{temp_col_name}_roll_mean_{rolling_window_hours}h'].shift(1)
                print(f"  Added rolling mean for {temp_col_name}")


        # Handle NaNs created by shift() and rolling().shift() operations
        # This is important BEFORE creating cyclical features from any columns that might be all NaN at the start
        initial_len = len(combined_df)
        combined_df.dropna(inplace=True) # Drop rows with NaNs from lags/rolling
        print(f"\nDropped {initial_len - len(combined_df)} rows due to NaNs from new lagged/rolling features.")
        if combined_df.empty:
            print("ERROR: DataFrame became empty after dropping NaNs from feature engineering. Check lag periods or data length.")
        else:
            # Cyclical features (useful for models that don't inherently understand circularity)
            print("\nAdding cyclical time features...")
            # Hour
            combined_df['hour_sin'] = np.sin(2 * np.pi * combined_df['hour_of_day'] / 24.0)
            combined_df['hour_cos'] = np.cos(2 * np.pi * combined_df['hour_of_day'] / 24.0)
            # Day of Week
            combined_df['day_of_week_sin'] = np.sin(2 * np.pi * combined_df['day_of_week'] / 7.0)
            combined_df['day_of_week_cos'] = np.cos(2 * np.pi * combined_df['day_of_week'] / 7.0)
            # Month
            combined_df['month_sin'] = np.sin(2 * np.pi * combined_df['month'] / 12.0)
            combined_df['month_cos'] = np.cos(2 * np.pi * combined_df['month'] / 12.0)
            # Day of Year
            combined_df['day_of_year_sin'] = np.sin(2 * np.pi * combined_df['day_of_year'] / 365.25)
            combined_df['day_of_year_cos'] = np.cos(2 * np.pi * combined_df['day_of_year'] / 365.25)
            print("Added cyclical time features.")
            # print(combined_df[['hour_of_day', 'hour_sin', 'hour_cos']].head(3)) # Sample check
            print(f"\nNew shape of combined_df after all feature engineering: {combined_df.shape}")
else:
    print("Combined DataFrame is empty. Skipping feature engineering.")


--- Feature Engineering: Adding Time-Based Features ---
Added basic time features (hour, day, month, year, etc.).

Adding lagged load features...
  Added lagged features for DOM_Load
  Added lagged features for PN_Load
  Added lagged features for PEPCO_Load
  Added lagged features for AECO_Load
  Added lagged features for PE_Load

Adding rolling mean weather features (e.g., Temp_C)...
  Added rolling mean for DOM_Temp_C
  Added rolling mean for PN_Temp_C
  Added rolling mean for PEPCO_Temp_C
  Added rolling mean for AECO_Temp_C
  Added rolling mean for PE_Temp_C

Dropped 168 rows due to NaNs from new lagged/rolling features.

Adding cyclical time features...
Added cyclical time features.

New shape of combined_df after all feature engineering: (29711, 94)


In [5]:
train_df, val_df, test_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame() # Initialize

if not combined_df.empty:
    print("\n--- Splitting Data into Train, Validation, and Test Sets ---")
    n = len(combined_df)
    if n < 24*7*4 : # Arbitrary check for minimum data length
        print(f"WARNING: Combined DataFrame has only {n} samples. Splits might be very small.")

    # Define split percentages
    train_split_pct = 0.7  # 70% for training
    val_split_pct = 0.15   # Next 15% for validation
    # Test split will be the remaining 15%
    
    train_end_idx = int(n * train_split_pct)
    val_end_idx = train_end_idx + int(n * val_split_pct) # Calculate val_end_idx based on train_end_idx
    
    train_df = combined_df.iloc[:train_end_idx].copy()
    val_df = combined_df.iloc[train_end_idx:val_end_idx].copy()
    test_df = combined_df.iloc[val_end_idx:].copy()
    
    print(f"Total samples in combined_df: {n}")
    if not train_df.empty:
        print(f"Training set shape:   {train_df.shape}, Index: {train_df.index.min()} to {train_df.index.max()}")
    else: print("Training set is empty.")
    if not val_df.empty:
        print(f"Validation set shape: {val_df.shape}, Index: {val_df.index.min()} to {val_df.index.max()}")
    else: print("Validation set is empty.")
    if not test_df.empty:
        print(f"Test set shape:       {test_df.shape}, Index: {test_df.index.min()} to {test_df.index.max()}")
    else: print("Test set is empty.")
    
    # Verify no overlap in indices if all sets are non-empty
    if not train_df.empty and not val_df.empty and not test_df.empty:
        assert train_df.index.max() < val_df.index.min(), "Train and Validation sets overlap!"
        assert val_df.index.max() < test_df.index.min(), "Validation and Test sets overlap!"
        print("Data splits are chronological and non-overlapping.")
    elif train_df.empty or val_df.empty or test_df.empty:
        print("Warning: One or more data splits are empty, cannot verify overlap fully.")
else:
    print("Combined DataFrame is empty. Skipping data splitting.")


--- Splitting Data into Train, Validation, and Test Sets ---
Total samples in combined_df: 29711
Training set shape:   (20797, 94), Index: 2022-01-08 05:00:00+00:00 to 2024-05-23 17:00:00+00:00
Validation set shape: (4456, 94), Index: 2024-05-23 18:00:00+00:00 to 2024-11-25 09:00:00+00:00
Test set shape:       (4458, 94), Index: 2024-11-25 10:00:00+00:00 to 2025-05-30 03:00:00+00:00
Data splits are chronological and non-overlapping.


In [6]:
train_scaled_df, val_scaled_df, test_scaled_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame() # Initialize
scaler = None # Initialize

if not train_df.empty and not val_df.empty and not test_df.empty : # Ensure all splits exist
    print("\n--- Scaling Numerical Features ---")
    
    # Use PJM_ZONES_UPPER from Cell 1 for consistency
    targets = [target_column_template.replace('{ZONE}', zone) for zone in PJM_ZONES_UPPER]
    
    # For simplicity, scale all columns. If you had true categorical string columns
    # (not 0/1 like 'IsDay' or WMO codes), they'd need one-hot encoding first and exclusion here.
    # Your current features (load, weather vars, time features like hour_sin) are all numeric.
    feature_columns_to_scale = train_df.columns.tolist()
    
    scaler = StandardScaler()
    
    # Fit the scaler ONLY on the training data
    train_scaled_data = scaler.fit_transform(train_df[feature_columns_to_scale])
    
    # Transform validation and test data using the FITTED scaler
    val_scaled_data = scaler.transform(val_df[feature_columns_to_scale])
    test_scaled_data = scaler.transform(test_df[feature_columns_to_scale])
    
    train_scaled_df = pd.DataFrame(train_scaled_data, columns=feature_columns_to_scale, index=train_df.index)
    val_scaled_df = pd.DataFrame(val_scaled_data, columns=feature_columns_to_scale, index=val_df.index)
    test_scaled_df = pd.DataFrame(test_scaled_data, columns=feature_columns_to_scale, index=test_df.index)
    
    print("Scaling complete.")
    # print("\nScaled Training Data (sample):")
    # print(train_scaled_df.head(3))
    # print("\nMean of scaled training data (sample of first 5 columns, should be ~0):")
    # print(train_scaled_df.mean().head())
    # print("\nStd Dev of scaled training data (sample of first 5 columns, should be ~1):")
    # print(train_scaled_df.std().head())

    if not os.path.exists(model_ready_data_dir):
        os.makedirs(model_ready_data_dir)
        print(f"Created directory: {model_ready_data_dir}")
        
    scaler_filename = 'feature_scaler_multiyear.joblib' # New name for new data
    scaler_path = os.path.join(model_ready_data_dir, scaler_filename)
    joblib.dump(scaler, scaler_path)
    print(f"\nScaler saved to: {scaler_path}")
    
else:
    print("One or more data splits (train_df, val_df, test_df) are empty. Skipping scaling.")


--- Scaling Numerical Features ---
Scaling complete.

Scaler saved to: D:\.Study\projects\EnergyForecasting\data\model_ready\feature_scaler_multiyear.joblib


In [7]:
if not train_scaled_df.empty and not val_scaled_df.empty and not test_scaled_df.empty:
    print("\n--- Saving Prepared DataFrames ---")
    
    if not os.path.exists(model_ready_data_dir): # Should have been created in cell 6 if needed
        os.makedirs(model_ready_data_dir)
        # print(f"Created directory: {model_ready_data_dir}") # Already printed if created

    # Use new filenames for the scaled data from multi-year processing
    train_scaled_df.to_csv(os.path.join(model_ready_data_dir, 'train_scaled_multiyear.csv'))
    val_scaled_df.to_csv(os.path.join(model_ready_data_dir, 'val_scaled_multiyear.csv'))
    test_scaled_df.to_csv(os.path.join(model_ready_data_dir, 'test_scaled_multiyear.csv'))
    
    print("Scaled train, validation, and test DataFrames saved with 'multiyear' suffix.")
    print(f"  Train: {os.path.join(model_ready_data_dir, 'train_scaled_multiyear.csv')}")
    print(f"  Val:   {os.path.join(model_ready_data_dir, 'val_scaled_multiyear.csv')}")
    print(f"  Test:  {os.path.join(model_ready_data_dir, 'test_scaled_multiyear.csv')}")

else:
    print("Scaled data (train_scaled_df, val_scaled_df, or test_scaled_df) is empty. Nothing to save.")


--- Saving Prepared DataFrames ---
Scaled train, validation, and test DataFrames saved with 'multiyear' suffix.
  Train: D:\.Study\projects\EnergyForecasting\data\model_ready\train_scaled_multiyear.csv
  Val:   D:\.Study\projects\EnergyForecasting\data\model_ready\val_scaled_multiyear.csv
  Test:  D:\.Study\projects\EnergyForecasting\data\model_ready\test_scaled_multiyear.csv
